In [12]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import os

# Define constants
img_width, img_height = 224, 224
train_data_dir = 'grayscale_dataset/train'
test_data_dir = 'grayscale_dataset/test'
epochs = 10
batch_size = 64

# Load pre-trained ResNet50 model
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Data augmentation for training
train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load training and testing data without augmentation
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='binary')

# Build a sequential model on top of ResNet50
model = Sequential()
model.add(resnet50_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Freeze the ResNet50 layers
for layer in resnet50_base.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
try:
    model.fit(train_generator,
              steps_per_epoch=train_generator.samples // batch_size,
              epochs=epochs,
              validation_data=test_generator,
              validation_steps=test_generator.samples // batch_size)
except Exception as e:
    print("Error occurred during training:", e)

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# Save the trained model
model.save('pcos_resnet50_model.h5')

Found 1924 images belonging to 2 classes.
Found 1922 images belonging to 2 classes.
Epoch 1/10
30/30 [==============================] - 190s 6s/step - loss: 0.7396 - accuracy: 0.7140 - val_loss: 0.2351 - val_accuracy: 0.9000
Epoch 2/10
30/30 [==============================] - 244s 8s/step - loss: 0.1652 - accuracy: 0.9656 - val_loss: 0.1320 - val_accuracy: 0.9828
Epoch 3/10
30/30 [==============================] - 253s 9s/step - loss: 0.1136 - accuracy: 0.9769 - val_loss: 0.0966 - val_accuracy: 0.9786
Epoch 4/10
30/30 [==============================] - 255s 9s/step - loss: 0.0884 - accuracy: 0.9839 - val_loss: 0.0808 - val_accuracy: 0.9906
Epoch 5/10
30/30 [==============================] - 252s 9s/step - loss: 0.0713 - accuracy: 0.9871 - val_loss: 0.0647 - val_accuracy: 0.9911
Epoch 6/10
30/30 [==============================] - 257s 9s/step - loss: 0.0606 - accuracy: 0.9887 - val_loss: 0.0539 - val_accuracy: 0.9901
Epoch 7/10
30/30 [==============================] - 259s 9s/step - los

c:\Users\kship\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
